In [ ]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [ ]:
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
# dom = BeautifulSoup(resp.text, 'html.parser')
# # for _ in dom.find_all('h3', {'class':'LC20lb'}):
# #     print(_.text.strip(), _.find_parents('a')[0]['href'])
# for _ in dom.select('div.r > a'):
#     print(_['href'], _.text.strip())

In [ ]:
url = 'https://search.naver.com/search.naver'
params = {
    'sm':'top_hty',
    'fbm':0,
    'ie':'utf8',
    'query':''
}
params['query'] = '파이썬'
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.content, 'html.parser')

In [ ]:
for _ in dom.find_all('ul', {'class':'type01'}):
    for a in [dt.find('a') for dt in _.find_all('dt')]:
        print(a['href'], a.text)

In [ ]:
for _ in dom.select('.type01 dt > a'):
    print(_['href'], _.text.strip())

In [ ]:
url = 'https://search.daum.net/search'
params = {
    'w':'tot',
    'DA':'YZR',
    't__nil_searchbox':'btn',
    'sug':'',
    'sugo':'',
    'sq':'',
    'o':'',
    'q':''
}
params['q'] = '파 이 썬'
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
# dom.find_all('ul', {'class':'list_info mg_cont clear'})
for _ in dom.find_all('div', {'class':'wrap_tit'}):
    print(_.find('a')['href'], _.find('a').text.strip())

In [ ]:
resp = download('http://pythonscraping.com/pages/page3.html')
dom = BeautifulSoup(resp.text, 'lxml')

In [ ]:
tag, #id, .class, .class.class.class

ul.class, ul.class.class.class
tag, tag, tag => CS

In [ ]:
dom.select_one('div#footer') == dom.select_one('#footer')

In [ ]:
dom.select_one('#footer').text.strip()

In [ ]:
Selector
tag1, tag2
tag1 tag2 => 자손 (find_all(recursive=True))
tag1 > tag2 => 자식 (file_all(recursive=False))
tag1 + tag2 => 형제(다음 노드) => tag2

In [ ]:
dom.select_one('#footer').find_parent().name

In [ ]:
len(dom.select('#wrapper > div')), len(dom.select('#wrapper > *'))

In [ ]:
[_.name for _ in dom.select('#wrapper > *')]

In [ ]:
dom.select_one('h1 + div').name

In [ ]:
dom.select_one('h1 + div').find_previous_sibling().name

In [ ]:
dom.select_one('body > div > h1 + div').name

In [ ]:
[_.text.strip() for _ in dom.select('.gift > td:nth-of-type(3)')]

In [ ]:
[_['src'].strip() for _ in dom.select('.gift > td > img')]

In [ ]:
url = 'http://example.webscraping.com/places/default/index'
resp = download(url)
dom = BeautifulSoup(resp.content, 'html.parser')

In [ ]:
urls.pop() 꺼낼, urls.append() 추가

In [ ]:
Queue => FIFO => pop(0)
Stack => LIFO => pop(-1)

In [ ]:
urls = list()
seen = list()
urls.append(url)

while urls: # Queue
    seed = urls.pop(0) # starting url
    seen.append(seed) # => 재방문 회피
    dom = BeautifulSoup(download(seed).text, 'html.parser') # HTTP
#     for _ in dom.select('a'): # extract hyperlinks
#         if _.has_attr('href'): # 나중에
#             if _['href'].startswith('/'): # filter1
#                 newUrls = urljoin(seed, _['href']) # Normalization
#                 # query부분 (GET방식에서 ? 이후에 나오는 파라미터 생략)
#                 if newUrls not in seen and newUrls not in urls: # 
#                     urls.append(newUrls)
# #                     print(newUrls)
    for _ in [_['href'] for _ in dom.select('a')
              if _.has_attr('href') and _['href'].startswith('/')]:
        newUrls = urljoin(seed, urlparse(_)[2])
        if newUrls not in seen and newUrls not in urls:
            urls.append(newUrls)
    print(len(urls), len(seen))

http => ht{2}p
https => https? => http, https
helllllllllo => hel{1,}o
다.                  그래서 => _+ => _
다. 그래서
[http]{3,4} => [h]tttp, [t]ttp, [p]pp => h | t | p
ttph, http, htp
h[t ]{1,}p
65:[A-Z] => [ABCDEFGHIZKLMNoPQR... Z] 대소문자 구별
97:[a-z] => [...]
[ㄱ-ㅎ] => 자음
[ㅏ-ㅣ] => 모음
[가-힣] => 음절
([가-힣]+)
^x => SQL(x%)
x$ => SQL(%x)
[^x] => not x
\b => 공백 + 문자 + 공백
\B => 문자 + ? + 문자
\b[A-Za-z]+\b => File Edit View Insert
\B[A-Za-z]+\B => F(il)e E(di)t V(ie)w I(nser)t
a href="javascript:~();#" => \W

In [ ]:
import re

In [ ]:
data = '''
park 800905-1049118
kim  700905-1059119
'''
pattern = re.compile('(\d{6})[-]\d{7}')
pattern.search(data)
print(pattern.sub('\g<1>-*******', data))
# data[6:20]

In [ ]:
re.match(r'Crow|Servo', 'HelloServoCrowHello')
re.search(r'^Crow|^Servo', 'HelloServoCrowHello')
re.search(r'Hello$', 'HelloServoCrowHello')

In [ ]:
re.search(r'(ABC)+', 'ABCABCA A').group()

In [ ]:
re.search(r'\bclass\b', 'no class at all')

In [ ]:
re.search(r'\Bclass\b', 'one subclass is')

In [ ]:
re.search(r'\Bclass\B', 'the declassified algorithm')

In [ ]:
result = re.search(r'(\w+) (\w+)', 'Issac Newton, physicist')
result.groups(), \
result.group(0), result.group(1), result.group(2)
# $@#%단어 단어
# \b(\w+)\b

In [ ]:
re.sub(r'(\w+) (\w+)', r'\g<2> \g<1>', 'Issac Newton, physicist')

In [ ]:
010-0000-0000 => 핸드폰번호

In [ ]:
data = '010-0001-0002'
data = '010 0000 0000'
data = '010.0000.0000'
data = '01000010002'
data = '0232901338'
result = re.search(r'(\d{1,3})[- .]?(\d{1,4})[- .]?(\d{1,4})', data)

In [ ]:
result.group(0), result.group(1), result.group(2), result.group(3)

In [ ]:
re.search(r'\d+[-]\d+[-]\d+', data)

In [ ]:
# data = '+82 2-0001-0002'
# data = '+82 10-0001-0002'
data = '+82 010-0001-0002'
# data = '82 010-0001-0002'
# ==> ??

In [ ]:
re.search(r'[+]?\d+.?\d+.?\d+.?\d+', data)

In [ ]:
resp = download('http://pythonscraping.com/pages/page3.html')
# dom = BeautifulSoup(resp.text, 'lxml')

In [ ]:
# re.search(r'([$]\d+[.]\d+)', resp.text).groups()
re.findall(r'[$]\d+[.]\d+', resp.text)

In [ ]:
re.findall(r'[$]\d+[.]\d+', resp.text)

In [ ]:
'../img/gifts/img1.jpg'

In [ ]:
for _ in re.findall(r'src="(.+?img\d[.]jpg)"', resp.text):
    print(urljoin(resp.request.url, _))

In [ ]:
'<span class="excitingNote">'

In [ ]:
re.findall(r'<(span) class="[^"]+">(.+)</\1>', resp.text)

In [ ]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
[_.strip() for _ in dom.find_all(text=re.compile(r'[.]\d+$'))]

In [ ]:
이메일
아이디@도메인
회사명.com|net|org|kr
    .co.kr|or.kr|go.kr|ac.kr

아이디 규칙:영어로시작, 특수문자X, -, _, 숫자

In [ ]:
data = '''
    test1@test.com
    test2@test.co.kr
    test3@email.test.com
    test4@email.test.co.kr
    12test@test.com
    한글@test.com
    test@test
'''

In [ ]:
re.findall(r'\b[A-Za-z][A-Za-z0-9-_.]+@[A-Za-z0-9-_]+(?:[.][A-Za-z0-9-_]+)+',
           data)

In [ ]:
re.findall(r'\b\w+@', data)

In [ ]:
http://도메인 => O
https://도메인 => O
    
ftp://도메인

In [ ]:
data = '''
    http://domain.com
    http://www.domain.com
    http://www.domain.co.kr
    https://www.domain.com
    https://www.domain.co.kr
    https://www.domain.co.kr/
    https://domain.co.kr/service
    https://www.domain.co.kr/service
    https://www.domain.co.kr/service/
    https://domain.co.kr/service?key=value
    https://www.domain.co.kr/service?key=value
    https://www.domain.co.kr/service?key=value/
    https://www.domain.co.kr/service/1/2?key=value
    https://www.domain.co.kr/service/1/2?key=value/
    ?key=value
    www.domain.com?key=value
    http://old.mail.domain.com?key=value
    http://domain.com#key=value
    http://www.domain.com?key=value
    http://www.domain.com#key=value
'''
# Query String (?이후부분) 빼고, URI(L)을 추출

In [ ]:
re.findall(r'https?://\w+(?:[./]\w+)+', data)

In [ ]:
re.findall(r'\b파이썬\B(\w+)\b', resp.text)

In [ ]:
re.findall(r'\b(\w+)\B파이썬\b', resp.text)

In [ ]:
\b파이썬\b
\b(\w+)\B파이썬\b
\b파이썬\B(\w+)\b

파이썬 파이썬 파이썬 = 3

~파이썬 파이썬~
은, 는, 이, 가~ => 명사
강이지를, 강이지가, 강아지는 => Unique
강아지+?

In [ ]:
()(강아지)(를) => 
()(강아지)(가) =>
()(강아지)(는) =>
(강아지)

In [ ]:
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')
urls = [{'url':_['href'], 'depth':1}
        for _ in dom.select('div.r > a[href]')]

In [ ]:
seen = list()

while urls: # Queue
    seed = urls.pop(-1)
    seen.append(seed)
    if seed['depth'] < 4: # 범위 제한 => Focused Crawling
        dom = BeautifulSoup(download(seed['url']).text, 'html.parser')
        for _ in [_['href'] for _ in dom.select('a[href]')
                  if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',
                              _['href'])]:
            newUrls = urljoin(seed['url'], urlparse(_)[2])
            if newUrls not in [_['url'] for _ in seen] \
            and newUrls not in [_['url'] for _ in urls]:
                urls.append({'url':newUrls, 'depth':seed['depth']+1})
        print(len(urls), len(seen))

In [ ]:
                  구글(검색어)-Queue(0)
    [1]     [2]    3    4     5    6    7     8 -> done
[1-1] 1-2 1-3 ..., 2-1, 2-2, ...
1-1-1 1-1-2 ... => leaf
=> BFS
                  구글(검색어)-Stack(-1)
    1     2    3    4     5    6    7     [8]
                                      8-1 8-2 [8-3] ...,
                                    8-3-1 [8-3-2] ... => leaf, done
=> DFS

In [ ]:
구글(파이썬) -> 8개 검색결과(depth=1) -> 다시 링크타고 이동(depth=2) -> depth=3

In [ ]:
temp = '/www.domain.co.kr/service/1/2?key=value/'
re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+', temp)

In [ ]:
url = 'https://search.naver.com/search.naver'
params = {
    'sm':'top_hty',
    'fbm':0,
    'ie':'utf8',
    'query':''
}
params['query'] = '파이썬'
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.content, 'html.parser')
urls = [{'url':_['href'], 'depth':1}
        for _ in dom.select('div.blog.section._blogBase._prs_blg dt > a[href]')]

In [ ]:
seen = list()

while urls: # Queue
    seed = urls.pop(0)
    seen.append(seed)
    if seed['depth'] < 3: # 범위 제한 => Focused Crawling (수정)
        dom = BeautifulSoup(download(seed['url']).text, 'html.parser')

        for _ in [_['href'] if _.has_attr('href') else _['src'] # 여기
                  for _ in dom.select('a[href], #mainFrame')
                  if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',
                _['href'] if _.has_attr('href') else _['src'])]: # 여기
            newUrls = urljoin(seed['url'], _) # 여기
            if urlparse(newUrls)[1] in ['blog.naver.com'] \
            and newUrls not in [_['url'] for _ in seen] \
            and newUrls not in [_['url'] for _ in urls]:# (수정)도메인 제한
                urls.append({'url':newUrls, 'depth':seed['depth']+1})
        print(len(urls), len(seen))

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect('pagerank.db')
cur = con.cursor()

In [ ]:
cur.executescript('''
    DROP TABLE IF EXISTS host;
    CREATE TABLE host(
        pk      INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc  TEXT NOT NULL
    );
    DROP TABLE IF EXISTS url;
    CREATE TABLE url(
        pk      INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        fk      INTEGER NOT NULL,
        ref     INTEGER NOT NULL,
        path    TEXT NOT NULL,
        crawled TEXT NOT NULL
    );
''')

In [ ]:
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')

for _ in dom.select('div.r > a[href]'):
    urlseg = urlparse(_['href'])
    # url => http://netloc/path?query
    # netloc = htttp://netloc
    netloc = '://'.join(urlseg[:2])
    # path = path?query
    path = urlseg[2]+('?'+urlseg[4] if urlseg[4] else '')
    # host테이블에 netloc이 있는지 확인
    cur.execute('SELECT pk FROM host WHERE netloc=? ORDER BY pk ASC LIMIT 0,1',
            ['://'.join(urlseg[:2]),])
    # 없다면,
    if not cur.fetchone():
        # host테이블에 netloc 추가
        cur.execute('INSERT INTO host(netloc) VALUES(?)',
                    ['://'.join(urlseg[:2]),])
        con.commit()
    # url테이블에 path 추가 (단, host테이블 netloc의 pk를 fk로 가져옴)
    cur.execute('''
    INSERT INTO url(fk, ref, path, crawled)
    VALUES((SELECT pk FROM host WHERE netloc=? ORDER BY pk LIMIT 0, 1), ?, ?, 'N')
    ''', [netloc, 0, path])
    con.commit()

In [ ]:
# while range(1000):#True:
for _ in range(100):
    cur.execute('''
        SELECT host.pk, url.pk, host.netloc, url.path FROM url
        INNER JOIN host ON host.pk=url.fk
        WHERE url.crawled='N'
        ORDER BY url.pk ASC
        LIMIT 0,1
    ''')
    seed = cur.fetchone()

    if not seed:
        break
        
    cur.execute('''
        UPDATE url SET crawled='Y' WHERE pk=?
    ''', [seed[1],])
    url = ''.join(seed[2:])
    dom = BeautifulSoup(download(url).text, 'html.parser')
    for _ in [_['href'] for _ in dom.select('a[href]')
              if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',
                          _['href'])]:
        urlseg = urlparse(urljoin(url, _))
        netloc = '://'.join(urlseg[:2])
        path = urlseg[2]+('?'+urlseg[4] if urlseg[4] else '')
        cur.execute('''
            SELECT pk FROM host WHERE netloc=?
            ORDER BY pk ASC LIMIT 0,1''', ['://'.join(urlseg[:2]),])
        if not cur.fetchone():
            cur.execute('INSERT INTO host(netloc) VALUES(?)',
                        ['://'.join(urlseg[:2]),])
            con.commit()
            
        cur.execute('''
            SELECT count(pk) FROM url WHERE fk=? and path=?
        ''', [seed[0], path])
        
        if cur.fetchone()[0] < 1:
            cur.execute('''
            INSERT INTO url(fk, ref, path, crawled)
            VALUES((SELECT pk FROM host WHERE netloc=? ORDER BY pk LIMIT 0, 1), ?, ?, 'N')
            ''', [netloc, seed[0], path])
            con.commit()
    break

In [ ]:
cur.execute('SELECT COUNT(pk) FROM host')
cnt = cur.fetchone()[0]

In [ ]:
cnt

In [ ]:
import numpy as np

In [ ]:
A = np.zeros((cnt+1,cnt+1))
v = np.zeros((cnt+1,))
S = np.zeros((cnt+1,))

In [ ]:
for j in range(1,cnt+1):
    cur.execute('''
        SELECT fk, count(fk) FROM url
        WHERE ref=? GROUP BY fk''', [j,])
    
    for i in cur:
        A[i[0],j] = i[1]
        S[j] += i[1]

In [ ]:
AA = A/S
AA[np.isnan(AA)] = 0

In [ ]:
v[:] = 1/cnt

In [ ]:
for _ in range(100):
    v = AA.dot(v)
    print('Iter:{}'.format(_+1))
    for _ in v.argsort()[::-1][:5]:
        cur.execute('SELECT * FROM host WHERE pk=?', [str(_),])
        print(cur.fetchone(), np.round(v[_], 10),
              A[:,_].sum(), A[_].sum())
    print()

In [ ]:
np.array([0])/np.array([0]), \
np.array([0])/np.array([1]), \
np.array([1])/np.array([0])

In [ ]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php'

params = {
    'id':'ppomppu',
    'page':1
}
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

In [ ]:
# Crawling[링크] + Scraping[내용]
# Focused Crawling - 게시판의 파라미터로 제한
for page in range(1,10):
    params['page'] = page
    resp = download(url, params, headers, 'GET')
    dom = BeautifulSoup(resp.text, 'lxml')
    
    # Scraping
    for _ in dom.select('#revolution_main_table tr[class^=list]')[1:]:
        item = dict()
        td = _.find_all('td', recursive=False)
        temp = title.findall(td[3].text)
        frag1 = temp[0] if len(temp) > 0 else td[3].text.strip()
        frag2 = td[-2].text.split('-')

        item['mall'] = frag1[0].strip()
        item['title'] = frag1[1].strip()
        item['price'] = frag1[2].strip()
        item['tu'] = frag2[0].strip() if len(frag2) > 1 else 0
        item['td'] = frag2[1].strip() if len(frag2) > 1 else 0
        print(item)
#         break

In [ ]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu'
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

urls = list()
seen = list()

urls.append(url)

while urls:
    seed = urls.pop(0)
    seen.append(seed)

    dom = BeautifulSoup(download(seed).text, 'lxml')
    for _ in [_['href']
              for _ in dom.select('''
              tr.list0 a[href], tr.list1 a[href], #page_list a[href]
              ''')
              if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+', _['href'])]:
        newUrls = urljoin(seed, _)
        if newUrls not in urls \
        and newUrls not in seen:
            urls.append(newUrls)
    print(len(urls), len(seen))

In [ ]:
# Scraping
for _ in dom.select('#revolution_main_table tr[class^=list]')[1:]:
    item = dict()
    td = _.find_all('td')
    frag1 = title.findall(td[3].text)[0]
    frag2 = td[-2].text.split('-')
    
    item['mall'] = frag1[0].strip()
    item['title'] = frag1[1].strip()
    item['price'] = frag1[2].strip()
    item['tu'] = frag2[0].strip() if len(frag2) > 1 else 0
    item['td'] = frag2[1].strip() if len(frag2) > 1 else 0
    print(item)
#     break

In [ ]:
title = re.compile(r'[[]?(\w+)[]]?([^(]+)[(](.+?)[)]')
price = re.compile(r'(.+?)[ /](.+?)')

In [ ]:
[[]? => (문자)[ 있고, 없고  \[
 (\w+) => 글자 1개 이상 (1)
 []]? => (문자)] 있고, 없고

([^(]+) => (그룹) (문자)(가 아닌 다른 문자가 1개 이상 (2)
  [(] => (문자)( 있고
    (.+?) => 아무글자나 1개 이상(Lazy)  (3)
    [)] => (문자)) 있고
    
[(1=???)] (2=???????) ((3=????))
[LG] 통돌이 모델명 (가격)

In [ ]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php'

params = {
    'id':'freeboard',
    'page':1
}
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

In [ ]:
urls = list()
seen = list()

urls.append({'url':url, 'depth':1})

while urls:
    seed = urls.pop(0)
    
    if seed['depth'] > 2:
        break
        
    resp = download(seed['url'])
    dom = BeautifulSoup(resp.text, 'lxml')
    
    seen.append(seed)
    
    for _ in dom.select('''
        .list0 > td:nth-of-type(3) > a,
        .list1 > td:nth-of-type(3) > a,
        #page_list a'''):
        if _['href'][0] in ['v', '/']:
            newurl = urljoin(url, _['href'])
            if newurl not in [_['url'] for _ in urls] and\
               newurl not in [_['url'] for _ in seen]:
                urls.append({'url':newurl,
                             'depth':seed['depth']+1})
            # 여기까지가 크롤링
            # 여기서부터가 스크래핑
            if dom.select_one('.board-contents') != None:
                print(dom.select_one('.board-contents').name)
    
# resp = download(url, params, headers)
# dom = BeautifulSoup(resp.text, 'lxml')
# for _ in dom.select('#page_list a'):
#     if _['href'].startswith('/'):
#         print(urljoin(url, _['href']))
#         resp = download(urljoin(url, _['href']))
#         dom = BeautifulSoup(resp.text, 'lxml')
#         if dom.select_one('.board-contents') != None:
#             print(dom.select_one('.board-contents').name)
#     print(_.text.strip())

In [ ]:
seen